In [1]:
from urllib.request import urlopen
from collections import OrderedDict
url = "https://sheets.googleapis.com/$discovery/rest?version=v4"

r = urlopen(url).read()
import json
j=json.loads(r.decode(), object_hook=OrderedDict)

def dump_l(l, indent=0):
    for item in l:
        if isinstance(item, list):
            dump_l(item, indent+1)
        elif isinstance(item, dict):
            dump_d(item, indent+1)
        else:
            print(indent*" ", item, sep="")
        
def dump_d(d, indent=0):
    for k, v in d.items():
        if isinstance(v, dict):
            print(indent*" ", k, ":", sep="")
            dump_d(v, indent+1)
        elif isinstance(v, list):
            dump_l(v, indent+1)
        else:
            assert isinstance(v, (int, str)), type(v)
            print(indent*" ", k, ': ', v, sep="")
            
def dump_resources(d, indent=0):
    for k, v in d.items():
        if k == 'resources':
            for k2, v2 in v.items():
                print(indent*" ", k2, ":", sep="")
                dump_resources(v2, indent+1)

In [2]:
dump_resources(j)

spreadsheets:
 values:
 sheets:


In [3]:
from goffice3.sheets import load_creds
from goffice3.sheets import GSession

In [4]:
SPREADSHEETS_URL_BASE = "https://sheets.googleapis.com/v4/spreadsheets/"

ujoin = "/".join
qjoin = "&".join

In [5]:
class Dimension():
    """ Dimension enum """
    DIMENSION_UNSPECIFIED = "DIMENSION_UNSPECIFIED"
    ROWS = "ROWS"
    COLUMNS = "COLUMNS"

class ValueRenderOption():
    """ ValueRenderOption enum """
    FORMATTED_VALUE = "FORMATTED_VALUE"
    UNFORMATTED_VALUE = "UNFORMATTED_VALUE"
    FORMULA = "FORMULA"
    
class DateTimeRenderOption():
    SERIAL_NUMBER = "SERIAL_NUMBER"
    FORMATTED_STRING = "FORMATTED_STRING"    
    
# brevity
D = Dimension
V = ValueRenderOption
DT = DateTimeRenderOption

class GSheetsAPI():
    """ This API layer implements the raw google sheets 
    v4 api via 1-1 translation from python to the REST
    api endpoint(s). 
    
    This api is defined as a hierarchy of collections in 
    the same way as the actual google api, to make 
    implementation directly reflect the google api structure. 
    
    All Api subobjects (spreadsheets, values...) retain reference
    to the top level api object as a just-in-case implemented in early
    programming. 
    """
    def __init__(self, session):
        self.session = session
        self.spreadsheets = SpreadsheetsAPICollection(session, self)
        
    @classmethod
    def from_creds(cls, creds):
        return cls(GSession(creds))
    
    
class SpreadsheetsAPICollection():
    def __init__(self, session, api):
        self.session = session
        self.api = api
        self.sheets = SheetsAPICollection(session, api)
        self.values = ValuesAPICollection(session, api)
        
    def batchUpdate(self, ssid, updates):
        raise NotImplementedError
        
    def create(self):
        raise NotImplementedError
        
    def get(self):
        raise NotImplementedError
        
        
class ValuesAPICollection():
    def __init__(self, session, api):
        self.session = session
        self.api = api

    def batchGet(self, ssid):
        raise NotImplementedError
    
    def batchUpdate(self, ssid):
        raise NotImplementedError
        
    def get(self, ssid, ss_range, major_dim=D.ROWS, value_render=V.FORMATTED_VALUE, datetime_render=DT.FORMATTED_STRING):
        query = qjoin(major_dim, value_render, datetime_render)
        url = ujoin(SPREADSHEETS_URL_BASE, str(ssid), "values", ss_range, query)
        return self.session.get(url)
    
    def update(self, ssid, ss_range):
        raise NotImplementedError
        
        
class SheetsAPICollection():
    def __init__(self, session, api):
        self.session = session
        self.api = api
    
    def copyTo(self):
        raise NotImplementedError

In [6]:
import os
os.chdir(os.path.expanduser("~/Documents/pyprojects/google_auths/.auth"))
c = GSheetsAPI(GSession(load_creds('_exp_auth_key.json')))

In [7]:
s = GSession(load_creds('_exp_auth_key.json'))

In [10]:
import json
dv3u = "https://www.googleapis.com/drive/v3"
dv3fileslist = dv3u+"/files"
r, c = s.get(dv3fileslist)
j=json.loads(c.decode())

1465539484.496945 goffice3.sheets.session DEBUG (60): Performing GET request: https://www.googleapis.com/drive/v3/files


In [11]:
j['files']

[{'id': '10x9ziXbfVm5ZcchAZjW1Zi7L0c0tDJfPhIvsnryvQZY',
  'kind': 'drive#file',
  'mimeType': 'application/vnd.google-apps.spreadsheet',
  'name': 'Slayer Tasks'},
 {'id': '0B85fKUTk3Dmbc3RhcnRlcl9maWxl',
  'kind': 'drive#file',
  'mimeType': 'application/pdf',
  'name': 'How to get started with Drive'}]